# OTROS TIEMPOS

In [15]:
weeks = 56

In [16]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "search",
    [[["inicio", ">=", FECHA_INICIO], ["inicio", "<=", FECHA_FIN]]]
)

lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "read",
    [lost_sales_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df = pd.DataFrame(lost_sales_data)

df['tarea'] = df['tarea_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

#Tarda  menos de 1 minuto

📅 Descargando registros desde 2024-10-02 hasta 2025-10-29


In [17]:
df.head(2)

,create_uid,__last_update,wk_line_id,create_date,name,tipo,tarea_id,delay_pausa,is_pause,finalizo,delay_parcial,delay_total,production_id,inicio,write_date,workcenter_id,costo_total,write_uid,id,delay_hours,display_name,tarea
0,"[345, Planta1]",2024-10-02 12:25:02,False,2024-10-02 11:50:02,"[609, MARIA PAULA PARRA PINILLA]",produccion,"[3, Recuperar cajas]",0.0,False,2024-10-02 07:25:02,0.0,0.0,False,2024-10-02 06:50:02,2024-10-02 12:25:02,False,7861.53,"[345, Planta1]",112,0.58,MARIA PAULA PARRA PINILLA,Recuperar cajas
1,"[345, Planta1]",2024-10-03 12:39:27,"[61916, EMBALAJE - DA2768 FILTRO AIRE KENWORTH, PETERBILT.]",2024-10-02 11:50:50,"[650, CLARA MARLEN OSPINA CASTRO]",sistema,"[2, Produccion]",89314.0,False,2024-10-03 07:39:27,13287.0,13287.0,"[11384, MO11336]",2024-10-02 06:50:51,2024-10-03 12:39:27,"[57, EMBALAJE]",7.38,"[345, Planta1]",113,3.69,CLARA MARLEN OSPINA CASTRO,Produccion


In [18]:
df_ot = df[df["tipo"]!="sistema"] #Se quitan las de op ya que no caen bien

df_ot.columns = df_ot.columns.str.lower()
df_ot = df_ot.rename(columns={
    "display_name": "operario",
    "inicio": "fecha_inicio",
    "tipo": "actividad",
})

# Convertir fechas
for col in ["fecha_inicio", "create_date", "__last_update"]:
    df_ot[col] = pd.to_datetime(df_ot[col], errors="coerce")

# Agregar columnas que no existen en df_ot para igualar estructura
df_ot["duracion_horas"] = df_ot["delay_hours"]
df_ot["centro_produccion"] = "OTRAS TAREAS"

df_ot = df_ot[["operario","__last_update","actividad","finalizo","fecha_inicio","tarea","duracion_horas"]]

# Hora actual (en tu zona horaria si quieres)
ahora = pd.Timestamp.now()

df_ot["fecha_inicio"] = pd.to_datetime(df_ot["fecha_inicio"], errors="coerce")
df_ot["finalizo"] = pd.to_datetime(df_ot["finalizo"], errors="coerce")

# --- 1️⃣ Crear columna en_curso ---
# True si no hay fecha de finalización
df_ot["en_curso"] = df_ot["finalizo"].isna() | (df_ot["finalizo"] == False)

# --- 2️⃣ Recalcular duración ---
df_ot["duracion_horas"] = df_ot.apply(
    lambda row: (
        (ahora - row["fecha_inicio"]).total_seconds() / 3600
        if row["en_curso"] and pd.notna(row["fecha_inicio"])
        else (row["finalizo"] - row["fecha_inicio"]).total_seconds() / 3600
        if pd.notna(row["finalizo"]) and pd.notna(row["fecha_inicio"])
        else 0
    ),
    axis=1
).round(2)

In [19]:
df_ot.tail(5)

,operario,__last_update,actividad,finalizo,fecha_inicio,tarea,duracion_horas,en_curso
34685,DAVID ESTEBAN MALAGON CETINA,2025-10-29 11:42:12,produccion,NaT,2025-10-29 06:42:12,Alistamiento MP,2.24,True
34687,JUAN DAVID GUTIERREZ CORTES,2025-10-29 11:42:55,produccion,NaT,2025-10-29 06:42:55,Molderia,2.23,True
34689,YEISON VILLANUEVA MELO,2025-10-29 11:45:11,produccion,2025-10-29 06:45:11,2025-10-29 06:44:59,Reproceso,0.00,False
34728,LUIS ENRIQUE MORA PEINADO,2025-10-29 13:49:01,produccion,2025-10-29 08:49:01,2025-10-29 08:40:12,Reproceso,0.15,False
34729,CLARA MARLEN OSPINA CASTRO,2025-10-29 13:48:48,produccion,2025-10-29 08:48:49,2025-10-29 08:40:28,Reproceso,0.14,False


# TIEMPOS POR OP

In [20]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd
import re

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

ops_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "search",
    [[["create_date", ">=", FECHA_INICIO], ["create_date", "<=", FECHA_FIN]]]
)

t_op = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "read",
    [ops_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df_op = pd.DataFrame(t_op)

df_op['Display_name'] = df_op['operario'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['OP'] = df_op['production_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['centro_produccion'] = df_op['workcenter_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))


df_op["nom_filtro"] = df_op["product_fabricado_id"].astype(str).apply(
    lambda x: re.search(r"\(([^)]+)\)", x).group(1) if re.search(r"\(([^)]+)\)", x) else None
)

df_op["product_ref"] = df_op["product_fabricado_id"].astype(str).apply(
    lambda x: re.search(r"\[([A-Z0-9]+)\]", x).group(1) if re.search(r"\[([A-Z0-9]+)\]", x) else None
)

df_op["maquina"] = df_op["fuerza_lab_id"].astype(str).apply(
    lambda x: re.search(r",\s*[\'\"]?([A-ZÁÉÍÓÚÑ0-9\s]+?)(?=\s*-\s*)", x, re.IGNORECASE).group(1).strip()
    if re.search(r",\s*[\'\"]?([A-ZÁÉÍÓÚÑ0-9\s]+?)(?=\s*-\s*)", x, re.IGNORECASE)
    else None
)



df_op = df_op.drop(columns=["operario", "production_id", "workcenter_id","write_uid","id","display_name","create_uid","wc_line_id","product_fabricado_id"])

#Tarda aprox 4 minutos


📅 Descargando registros desde 2024-10-02 hasta 2025-10-29


In [21]:
# --- Limpieza df_op ---
df_op.columns = df_op.columns.str.lower()
df_op = df_op.rename(columns={"evento": "actividad", "display_name": "operario"})

# Asegurar tipos
for col in ["fecha_evento", "create_date", "write_date", "__last_update"]:
    df_op[col] = pd.to_datetime(df_op[col], errors="coerce")


In [22]:
from datetime import timedelta

# Asegurar tipos de fecha correctos
df_op["fecha_evento"] = pd.to_datetime(df_op["fecha_evento"], errors="coerce")

# --- 1️⃣ Crear columna 'final' ---
df_op["final"] = df_op.apply(
    lambda row: row["fecha_evento"] + timedelta(seconds=row["segundos_evento"])
    if pd.notna(row["segundos_evento"]) else row["fecha_evento"],
    axis=1
)

# --- 2️⃣ Ordenar por operario y fecha_evento ---
df_op = df_op.sort_values(["operario", "fecha_evento"])

# --- 3️⃣ Marcar el último evento por operario ---
df_op["ultimo_evento"] = df_op.groupby("operario")["fecha_evento"].transform("max")

# --- 4️⃣ Crear columna en_curso ---
# En curso = el evento más reciente del operario y su actividad es "iniciar"
df_op["en_curso"] = (df_op["fecha_evento"] == df_op["ultimo_evento"]) & (df_op["actividad"] == "iniciar")

# --- 5️⃣ Filtrar: eliminar 'iniciar' antiguos (solo mantener el último 'iniciar' si está en curso) ---
df_op_filtrado = df_op[
    (df_op["actividad"] != "iniciar") |  # mantener todo lo que no sea iniciar
    (df_op["en_curso"])                  # o mantener el último iniciar de cada operario
].copy()

# --- 6️⃣ Limpieza final ---
df_op_filtrado = df_op_filtrado.drop(columns=["ultimo_evento"])


In [23]:
#df_op_filtrado[df_op_filtrado["en_curso"]==True]
df_op_filtrado[(df_op_filtrado["operario"]=="JAMER PUPO SIERRA")& (df_op_filtrado["fecha_evento"]>"2025-10-21")].head(2)

,__last_update,horas_evento,create_date,fecha_evento,fuerza_lab_id,actividad,segundos_evento,write_date,operario,op,centro_produccion,nom_filtro,product_ref,maquina,final,en_curso
101898,2025-10-21 12:10:37,0.55,2025-10-21 12:10:37,2025-10-21 07:10:37,"[113786, ENTUBADORA 3 - DA8145 FILTRO AIRE EXCAVADORA RHINO REX210]",finalizar,1997.0,2025-10-21 12:10:37,JAMER PUPO SIERRA,MO13349,CORTE,025 DA8145,DAB08145025,ENTUBADORA 3,2025-10-21 07:43:54,False
101939,2025-10-21 12:43:35,0.52,2025-10-21 12:43:35,2025-10-21 07:43:35,"[113804, ENTUBADORA 3 - DA8169 FILTRO AIRE COSECHADORAS WORL]",finalizar,1864.0,2025-10-21 12:43:35,JAMER PUPO SIERRA,MO13352,CORTE,025 DA8169,DAB08169025,ENTUBADORA 3,2025-10-21 08:14:39,False


## UNIR DATAFRAMES

In [24]:
#df_union.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/control_operarios.xlsx")

In [25]:
df_op_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52303 entries, 77 to 104518
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   __last_update      52303 non-null  datetime64[ns]
 1   horas_evento       52303 non-null  float64       
 2   create_date        52303 non-null  datetime64[ns]
 3   fecha_evento       52303 non-null  datetime64[ns]
 4   fuerza_lab_id      52303 non-null  object        
 5   actividad          52303 non-null  object        
 6   segundos_evento    52303 non-null  float64       
 7   write_date         52303 non-null  datetime64[ns]
 8   operario           52303 non-null  object        
 9   op                 52303 non-null  object        
 10  centro_produccion  52303 non-null  object        
 11  nom_filtro         52301 non-null  object        
 12  product_ref        52205 non-null  object        
 13  maquina            43902 non-null  object        
 14  final    

In [26]:
df_ot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6125 entries, 0 to 34729
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   operario        6125 non-null   object        
 1   __last_update   6125 non-null   datetime64[ns]
 2   actividad       6125 non-null   object        
 3   finalizo        6123 non-null   datetime64[ns]
 4   fecha_inicio    6125 non-null   datetime64[ns]
 5   tarea           6125 non-null   object        
 6   duracion_horas  6125 non-null   float64       
 7   en_curso        6125 non-null   bool          
dtypes: bool(1), datetime64[ns](3), float64(1), object(3)
memory usage: 388.8+ KB


In [27]:
# Copias de trabajo
df1 = df_op_filtrado.copy()
df2 = df_ot.copy()

# --- Homologar df_op_filtrado a estructura de df_ot ---
df1 = df1.rename(columns={
    'fecha_evento': 'fecha_inicio',
    'final': 'finalizo',
    'horas_evento': 'duracion_horas',
    'centro_produccion': 'tarea'
})

# --- Agregar columna de origen ---
df1['origen'] = 'produccion'
df2['origen'] = 'otras_tareas'

# --- Crear columna de segundos_evento ---
# En producción ya existe, pero renombramos para coherencia
if 'segundos_evento' not in df1.columns:
    df1['segundos_evento'] = df1['duracion_horas'] * 3600
# En tareas la calculamos desde duracion_horas
df2['segundos_evento'] = df2['duracion_horas'] * 3600

# --- Unir ambos sin perder datos ---
df_union = pd.concat([df1, df2], ignore_index=True, sort=False)

# --- Ordenar columnas ---
cols_base = [
    'operario', '__last_update', 'actividad', 'fecha_inicio', 'finalizo',
    'tarea', 'duracion_horas', 'segundos_evento', 'en_curso', 'origen'
]

cols_extra = [
    'op', 'fuerza_lab_id', 'maquina', 'nom_filtro', 'product_ref',
    'create_date', 'write_date'
]

cols_final = cols_base + [c for c in cols_extra if c in df_union.columns]
df_union = df_union[cols_final]


if 'tarea' in df_union.columns:
    df_union['tarea'] = df_union['tarea'].astype(str).str.upper()


# Tomar fecha y hora actual
ahora = pd.Timestamp.now()

# Filtrar filas en curso
mask_en_curso = df_union['en_curso'] == True

# Calcular diferencia de tiempo
df_union.loc[mask_en_curso, 'duracion_horas'] = (
    (ahora - df_union.loc[mask_en_curso, 'fecha_inicio']).dt.total_seconds() / 3600
)

df_union.loc[mask_en_curso, 'segundos_evento'] = (
    df_union.loc[mask_en_curso, 'duracion_horas'] * 3600
)

# Asegurar que la columna sea texto y eliminar espacios
df_union['op'] = df_union['op'].astype(str).str.strip()

df_union['operario'] = df_union['operario'].astype(str).str.strip()

df_union.loc[df_union['actividad'].str.lower().str.strip() == 'produccion', 'actividad'] = 'otras_tareas'

# Guardar a Excel
df_union.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/operarios_tiempo_completo.xlsx", index=False)

#TARDA 1 MINUTO


In [28]:
df_union[df_union["en_curso"]==True].head()

,operario,__last_update,actividad,fecha_inicio,finalizo,tarea,duracion_horas,segundos_evento,en_curso,origen,op,fuerza_lab_id,maquina,nom_filtro,product_ref,create_date,write_date
2422,ALVARO MELO PRADA,2025-10-29 13:32:48,iniciar,2025-10-29 08:32:48,2025-10-29 08:32:48,CORTE,0.518345,1866.041301,True,produccion,MO13380,"[114522, TAPAS - DA2980 FILTRO AIRE HYUNDAI HD270]",TAPAS,025 DA2980,DAR02980025,2025-10-29 13:32:48,2025-10-29 13:32:48
3981,BLANCA CECILIA AVELLA ARIAS,2025-10-29 11:41:32,iniciar,2025-10-29 06:41:32,2025-10-29 06:41:32,INYECCION,2.372789,8542.041301,True,produccion,MO13308,"[114336, OMS - DA4570A FILTRO AIRE 2_ DONSSON - PERKINS, CATERPILLAR]",OMS,025 DA4570A,DAB14570025,2025-10-29 11:41:32,2025-10-29 11:41:32
5858,BLANCA DORA CUBIDES SANCHEZ,2025-10-29 13:08:56,iniciar,2025-10-29 08:08:56,2025-10-29 08:08:56,CORTE,0.916123,3298.041301,True,produccion,MO13402,"[114513, ENTUBADORA 3 - DA4728 FILTRO AIRE 2_ TEREX]",ENTUBADORA 3,025 DA4728,DAB04728025,2025-10-29 13:08:56,2025-10-29 13:08:56
10206,CLARA MARLEN OSPINA CASTRO,2025-10-29 13:48:48,iniciar,2025-10-29 08:48:48,2025-10-29 08:48:48,EMBALAJE,0.251678,906.041301,True,produccion,MO13361,"[114524, EMBALAJE - DA8007 FILTROAIRE PERKINS, CAT]",EMBALAJE,025 DA8007,DAB08007025,2025-10-29 13:48:48,2025-10-29 13:48:48
11428,DANILO PUPO SIERRA,2025-10-29 11:41:12,iniciar,2025-10-29 06:41:12,2025-10-29 06:41:12,PLISADO,2.378345,8562.041301,True,produccion,MO13393,"[114354, PLISADORA 4 - DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.]",PLISADORA 4,025 DA2570,DAB02570025,2025-10-29 11:41:12,2025-10-29 11:41:12


In [29]:
#df_union[(df_union["operario"]=="JAMER PUPO SIERRA")& (df_union["fecha_inicio"]>"2025-10-21")].head(7)